<a href="https://colab.research.google.com/github/GabrielKonno/AutoMLPratica/blob/main/Auto_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

x = housing.data
y = housing.target

In [15]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
!pip install autokeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 79.6 MB/s eta 0:00:00


In [7]:
import autokeras as ak
print(ak.__version__)


2.0.0


In [8]:
!pip install autokeras --upgrade


In [18]:
import autokeras as ak

# Definir o nó de entrada genérico
input_node = ak.Input()

# Definir o nó de saída para regressão
output_node = ak.RegressionHead(
    loss='mean_squared_error',
    metrics=['mean_squared_error']
)

# Criar o modelo automático usando AutoModel
automl = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=4,
    objective='val_mean_squared_error',
    overwrite=True,
    seed=42
)


In [19]:
automl.fit(xtrain, ytrain, epochs=2)

Trial 4 Complete [00h 00m 05s]
val_mean_squared_error: 1.4265239238739014

Best val_mean_squared_error So Far: 1.4265239238739014
Total elapsed time: 00h 00m 23s
Epoch 1/2
516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 31413.5078 - mean_squared_error: 31413.5078
Epoch 2/2
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4.8936 - mean_squared_error: 4.8936


In [22]:
import pandas as pd

# Acessar o tuner
tuner = automl.tuner

# Acessar os trials
trials = tuner.oracle.trials

# Criar uma lista para armazenar os dados dos trials
trial_data = []

for trial_id, trial in trials.items():
    # Obter a métrica de interesse do trial
    score = trial.score  # Pode ser None em alguns casos
    # Se o score for None, você pode acessar o valor da melhor métrica
    if score is None:
        # Substitua 'val_mean_squared_error' pela sua métrica de interesse
        score = trial.metrics.get_best_value('val_mean_squared_error')

    # Obter os hiperparâmetros usados no trial
    hparams = trial.hyperparameters.values
    # Armazenar os dados
    trial_data.append({
        'Trial ID': trial_id,
        'Score': score,
        'Hyperparameters': hparams
    })

# Criar um DataFrame com os resultados
leaderboard = pd.DataFrame(trial_data)

# Ordenar a tabela pelos scores (ajuste ascending conforme necessário)
leaderboard = leaderboard.sort_values(by='Score', ascending=True).reset_index(drop=True)

# Exibir a leaderboard
print(leaderboard)


  Trial ID     Score                                    Hyperparameters
0        3  1.426524  {'general_block_1/dense_block_1/use_batchnorm'...
1        1  1.499391  {'general_block_1/dense_block_1/use_batchnorm'...
2        2  1.620332  {'general_block_1/dense_block_1/use_batchnorm'...
3        0  8.446736  {'general_block_1/dense_block_1/use_batchnorm'...


In [27]:
import numpy as np

ypred = automl.predict(xtest)
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(ytest, ypred)
rmse = np.sqrt(mse)


print("MSE:", mse)
print("RMSE:", rmse)

 77/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
MSE: 1.3593590579752735
RMSE: 1.1659155449582417
